## Preparation

In [1]:
# Import dependencies

!pip install opendatasets
!pip install pandas
!pip install kaggle
!pip install unrar
!pip install patool
import tensorflow as tf
import opendatasets as od
import pandas as pd
import numpy as np
import cv2 as cv
from tensorflow.keras import datasets, layers, models, optimizers
import matplotlib.pyplot as plt
import matplotlib.image as img
import os
import patoolib
from PIL import Image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 1.9 MB/s eta 0:00:00


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
# Initialize paths
data_dir = os.path.join(os.curdir, 'drive', 'MyDrive', 'Muvision', 'datasets', 'handwritten_math_symbols')
model_dir = os.path.join(os.curdir, 'drive', 'MyDrive', 'Muvision', 'models')

custom_model_path = os.path.join(model_dir, 'custom_model.h5')
resnet_model_path = os.path.join(model_dir, 'resnet_model.h5')
vgg19_model_path = os.path.join(model_dir, 'vgg19_model.h5')
mobilenet_model_path = os.path.join(model_dir, 'mobilenet_model.h5')
ensemble_model_path = os.path.join(model_dir, 'ensemble_model.h5')

In [ ]:
# Initialize important variables
classes = 19
input_shape=(45,45,3)
batch_size = 1
image_size = (45, 45)

## Preprocess the colors


In [ ]:
# Let's see what an individual image in the dataset looks like.
sample_img_path = os.path.join(data_dir, '0', '10014.jpg')
img = np.asarray(Image.open(sample_img_path))
imgplot = plt.imshow(img)
print(repr(img))
print(img.shape)

In [ ]:
# Since the images are already gray scale, we don't need to make any changes.

## Preprocess data set

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data_set = tf.keras.utils.image_dataset_from_directory('/content/drive/MyDrive/Muvision/datasets/handwritten_math_symbols', batch_size = batch_size, image_size = image_size)

In [ ]:
# Normalize images
data_set = data_set.map(lambda x,y: (x/255, y))

In [ ]:
print(type(data_set))

In [ ]:
# Allocate training, validation, and test sizes
train_size = int(len(data_set)* .7)
val_size = int(len(data_set)*.2)+1
test_size = int(len(data_set)*.1)+1

In [ ]:
# Split up dataset into train, validation and test
train = data_set.take(train_size)
val = data_set.skip(train_size).take(val_size)
test = data_set.skip(train_size + val_size).take(test_size)

## Create Custom Model using Tensorflow

In [ ]:
custom_model = models.Sequential(name='Custom_Model')

# Convolutional base
custom_model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
custom_model.add(layers.MaxPooling2D((2, 2)))
custom_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
custom_model.add(layers.MaxPooling2D((2, 2)))
custom_model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# Dense layers
custom_model.add(layers.Flatten())
custom_model.add(layers.Dense(64, activation='relu'))
custom_model.add(layers.Dense(classes, activation='softmax'))
custom_model.summary()

# The summary shows that the convolutional base has a (4, 4, 64) output, which
#   is flattened into a (1024) shaped vector, and then sent through two Dense
#   layers

custom_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

custom_model_history = custom_model.fit(train, epochs=10,
                         validation_data=val)

custom_model.save(custom_model_path)

In [ ]:
plt.plot(custom_model_history.history['accuracy'], label='accuracy')
plt.plot(custom_model_history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = custom_model.evaluate(test, verbose=2)

## Create Model using Resnet-50

In [ ]:
resnet_model = models.Sequential(name='Resnet_Model')

resnet_pretrained_model= tf.keras.applications.ResNet50(include_top=False,

                   input_shape=input_shape,

                   pooling='max',classes=classes,

                   weights='imagenet')

for each_layer in resnet_pretrained_model.layers:

        each_layer.trainable=False

resnet_model.add(resnet_pretrained_model)

94765736/94765736 [==============================] - 3s 0us/step


In [ ]:
resnet_model.add(layers.Flatten())

resnet_model.add(layers.Dense(512, activation='relu'))
resnet_model.add(layers.Dense(classes, activation='softmax'))

In [ ]:
resnet_model.compile(optimizer = optimizers.Adam(learning_rate=0.001),loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

epochs = 10
history = resnet_model.fit(train, validation_data=val, epochs=epochs)
resnet_model.save(resnet_model_path)

Epoch 1/10
7049/7049 [==============================] - 426s 60ms/step - loss: 1.3445 - accuracy: 0.5795 - val_loss: 0.7877 - val_accuracy: 0.7444
Epoch 2/10
7049/7049 [==============================] - 411s 58ms/step - loss: 0.8693 - accuracy: 0.7153 - val_loss: 0.6582 - val_accuracy: 0.7881
Epoch 3/10
7049/7049 [==============================] - 411s 58ms/step - loss: 0.7243 - accuracy: 0.7666 - val_loss: 0.6750 - val_accuracy: 0.7841
Epoch 4/10
7049/7049 [==============================] - 415s 59ms/step - loss: 0.6475 - accuracy: 0.7885 - val_loss: 0.6463 - val_accuracy: 0.7896
Epoch 5/10
7049/7049 [==============================] - 407s 58ms/step - loss: 0.5963 - accuracy: 0.8071 - val_loss: 0.7565 - val_accuracy: 0.7906
Epoch 6/10
7049/7049 [==============================] - 401s 57ms/step - loss: 0.5612 - accuracy: 0.8173 - val_loss: 0.8775 - val_accuracy: 0.7464
Epoch 7/10
7049/7049 [==============================] - 396s 56ms/step - loss: 0.5209 - accuracy: 0.8323 - val_loss: 0

In [ ]:
plt.figure(figsize=(8, 8))

epochs_range= range(epochs)

plt.plot( epochs_range, history.history['accuracy'], label="Training Accuracy")

plt.plot(epochs_range, history.history['val_accuracy'], label="Validation Accuracy")

plt.axis(ymin=0.4,ymax=1)

plt.grid()

plt.title('Model Accuracy')

plt.ylabel('Accuracy')

plt.xlabel('Epochs')

plt.legend(['train', 'validation'])

In [ ]:
#plotter_lib.show()

plt.savefig('output-plot.png')

## Create Model using VGG19

In [ ]:
from tensorflow.keras.applications import VGG19

In [ ]:
vgg19_model = models.Sequential(name='VGG19_Model')

vgg19_pretrained_model= tf.keras.applications.VGG19(include_top=False,

                   input_shape=input_shape,

                   pooling='max',classes=classes,

                   weights='imagenet')

for each_layer in vgg19_pretrained_model.layers:
        each_layer.trainable=False

vgg19_model.add(vgg19_pretrained_model)

In [ ]:
vgg19_model.add(layers.Flatten())

vgg19_model.add(layers.Dense(512, activation='relu'))

vgg19_model.add(layers.Dense(classes, activation='softmax'))

In [ ]:
vgg19_model.compile(optimizer=optimizers.Adam(learning_rate=0.001),loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

epochs = 8
try:
  history = vgg19_model.fit(train, validation_data=val, epochs=epochs)
except Exception as e: print(e)

vgg19_model.save(vgg19_model_path)

In [ ]:
# Test

plt.figure(figsize=(8, 8))

epochs_range= range(epochs)

plt.plot( epochs_range, history.history['accuracy'], label="Training Accuracy")

plt.plot(epochs_range, history.history['val_accuracy'], label="Validation Accuracy")

plt.axis(ymin=0.4,ymax=1)

plt.grid()

plt.title('Model Accuracy')

plt.ylabel('Accuracy')

plt.xlabel('Epochs')

plt.legend(['train', 'validation'])

## Create Model using MobileNet

In [ ]:
from tensorflow.keras.applications import MobileNetV2

In [ ]:
mobilenet_model = models.Sequential(name='MobileNet_Model')
mobilenet_pretrained_model = tf.keras.applications.MobileNetV2(include_top=False,
                   input_shape=input_shape,
                   pooling='max',classes=classes,
                   weights='imagenet')
for each_layer in mobilenet_pretrained_model.layers:
        each_layer.trainable=False

mobilenet_model.add(mobilenet_pretrained_model)

In [ ]:
mobilenet_model.add(layers.Flatten())

mobilenet_model.add(layers.Dense(512, activation='relu'))

mobilenet_model.add(layers.Dense(classes, activation='softmax'))

In [ ]:
mobilenet_model.compile(optimizer=optimizers.Adam(learning_rate=0.001),loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

epochs = 8
try:
  history = mobilenet_model.fit(train, validation_data=val, epochs=epochs)
except Exception as e: print(e)
mobilenet_model.save(mobilenet_model_path)

In [ ]:
# Test

plt.figure(figsize=(8, 8))

epochs_range= range(epochs)

plt.plot(epochs_range, history.history['accuracy'], label="Training Accuracy")

plt.plot(epochs_range, history.history['val_accuracy'], label="Validation Accuracy")

plt.axis(ymin=0.4,ymax=1)

plt.grid()

plt.title('Model Accuracy')

plt.ylabel('Accuracy')

plt.xlabel('Epochs')

plt.legend(['train', 'validation'])

## Ensemble

In [ ]:
# Load trained models from file system
models = [models.load_model(custom_model_path),
          models.load_model(resnet_model_path),
          models.load_model(vgg19_model_path),
          models.load_model(mobilenet_model_path)]

In [ ]:
# Create the weighted average ensemble model
weights = [0.25, 0.25, 0.25, 0.25]
inputs = layers.Input(shape = input_shape)
outputs = [model(inputs) for model in models]
weighted_average = tf.reshape((), (0, classes)) # An empty tensor

for i in range(0, len(models)):
    if i == 0:
        weighted_average = weights[i] * outputs[i]
    else:
        weighted_average = weighted_average + weights[i] * outputs[i]
    # weighted_average = weighted_average + weights[i] * outputs[i]
    # weighted_average = tf.math.add(weighted_average, weights[i] * outputs[i])
# weighted_average = weights[0]*outputs[0] + weights[1]*outputs[1] + weights[2]*outputs[2] + weights[3]*outputs[3]

ensemble_model = tf.keras.Model(inputs = inputs, outputs = weighted_average, name = 'Ensemble_Model')

In [ ]:
ensemble_model.compile(optimizer = optimizers.Adam(learning_rate=0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
ensemble_model.save(ensemble_model_path)

In [ ]:
ensemble_model.evaluate(test)

## Testing

In [ ]:
# Testing the ensemble model

iterator = test.__iter__()
current = iterator.get_next()
plt.imshow(current[0][0])

In [ ]:
ensemble_model_prediction = ensemble_model.predict(current[0])
print(ensemble_model_prediction[0])

In [ ]:
custom_model = tf.keras.models.load_model(custom_model_path)
custom_model_prediction = custom_model.predict(current[0])
print(custom_model_prediction[0])

In [ ]:
resnet_model = tf.keras.models.load_model(resnet_model_path)
resnet_model_prediction = resnet_model.predict(current[0])
print(resnet_model_prediction[0])

In [ ]:
vgg19_model = tf.keras.models.load_model(vgg19_model_path)
vgg19_model_prediction = vgg19_model.predict(current[0])
print(vgg19_model_prediction[0])

In [ ]:
mobilenet_model = tf.keras.models.load_model(mobilenet_model_path)
mobilenet_model_prediction = mobilenet_model.predict(current[0])
print(mobilenet_model_prediction[0])

In [ ]:
custom_model.evaluate(test)

In [ ]:
mobilenet_model.evaluate(test)
resnet_model.evaluate(test)
vgg19_model.evaluate(test)

## Creating a function to classify images

In [ ]:
import tensorflow as tf
import numpy as np
import os
from PIL import Image
from skimage import transform

ensemble_model_path = os.path.join(os.curdir, 'drive', 'MyDrive', 'Muvision', 'models', 'custom_model2.h5')
ensemble_model = tf.keras.models.load_model(ensemble_model_path)
image_size = (45,45)

In [ ]:
from skimage import transform
from skimage import morphology
def classify(np_image):
  # np_image = Image.open(filename)
  # print(shape(np_image))
  #  np_image = tf.image.rgb_to_grayscale(np_image, name='jeff')
  # np_image = cv.cvtColor(image, cv.COLOR_RGBA2RGB)
  np_image = np.array(np_image).astype('float32')/256
  np_image = transform.resize(np_image, (45, 45,3))
  imgplot = plt.imshow(np_image)
  plt.show()
  np_image = np.expand_dims(np_image, axis=0)
  prediction = ensemble_model.predict(np_image)
  # class_order = ['!', '(', ')', '+', 'forward_slash', '-', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '=', 'A', 'C', 'Delta', 'G', 'H', 'M', 'N', 'R', 'S', 'T', 'X', '[', ']', 'alpha', 'ascii_124', 'b', 'beta', 'cos', 'd', 'div', 'e', 'exists', 'f', 'forall', 'forward_slash', 'gamma', 'geq', 'gt', 'i', 'in', 'infty', 'int', 'j', 'k', 'l', 'lambda', 'ldots', 'leq', 'lim', 'log', 'lt', 'mu', 'neq', 'o', 'p', 'phi', 'pi', 'pm', 'prime', 'q', 'rightarrow', 'sigma', 'sin', 'sqrt', 'sum', 'tan', 'theta', 'times', 'u', 'v', 'w', 'y', 'z', '{', '}']
  class_order = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '+', '.', '\\div', '=', '\\times', '-', 'x', 'y', 'z']
  return np.argmax(prediction)
  res = tf.math.argmax(prediction[0])
  # return int(tensor)
  return class_order[int(tensor)]

In [ ]:
sample_img_name = os.path.join(os.curdir, 'drive', 'MyDrive', 'Muvision', 'sample_img', 'x.png')
image = Image.open(sample_img_name)
print(image)
print(classify(image))